<a href="https://colab.research.google.com/github/Omkar-here/Agri-Bot/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.5 MB/s eta 0:13:04tcmalloc: large alloc 1147494400 bytes == 0x399f6000 @  0x7f678f66a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.5 MB/s eta 0:10:24tcmalloc: large alloc 1434370048 bytes == 0x7e04c000 @  0x7f678f66a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('I dont like you', return_tensors='pt')

In [ ]:
tokens

tensor([[  101,   151, 11930, 11531, 10855,   102]])

In [ ]:
result = model(tokens)
result


SequenceClassifierOutput([('logits',
                           tensor([[ 0.9906,  1.1298,  0.6482, -0.9291, -1.5111]],
                                  grad_fn=<AddmmBackward>))])

In [ ]:
int(torch.argmax(result.logits))+1

2

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews


['Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 'I went here a little while ago- a beautiful morning,a lovely little brew house on a quaint street corner- perfection.I went to this cafe with my step-daughter Lucille.She was always raving about how great it was to her mother, so I thought it would be a nice idea to 

In [ ]:

import numpy as np
import pandas as pd


In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [ ]:
df['review'].iloc[0]

'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,Great staff and food. Must try is the pan fri...,5
1,I came to Social brew cafe for brunch while ex...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,I went here a little while ago- a beautiful mo...,2
4,We came for brunch twice in our week-long visi...,4
5,Ron & Jo are on the go down under and Wow! We...,5
6,Great coffee and vibe. That's all you need. C...,5
7,Great coffee and vibe. That's all you need. C...,4
8,Good coffee and toasts. Straight up and down -...,5
9,This place is a gem. The ambiance is to die fo...,3


In [ ]:
df['review'].iloc[3]

'I went here a little while ago- a beautiful morning,a lovely little brew house on a quaint street corner- perfection.I went to this cafe with my step-daughter Lucille.She was always raving about how great it was to her mother, so I thought it would be a nice idea to go here with her for her birthday... boy was I wrong.She announced her hatred for me while I was waiting for my extra large iced frappé. It felt like hours of awkward silence once she said those four words; "you\'re a low-life."Was it in my mind, or was my drink taking ages to arrive? The hands on the clock didn\'t budge from the last time I glanced at them- 7:43AM, where the fuck is my drink?"Why do you always feel you have to be my friend? You\'re not my dad!" She fired.I could only sit there, my head facing down towards the floral tablecloth that lay beneath my quivering arms. The bullet lodged in my heart.I don\'t understand why she hates me so much; is it my jokes? The funny way I walk? The fact that I often scream my